# FIFA 21 Complete Player Dataset: Cleaning & Merging

This notebook processes the **FIFA 21 Complete Player Dataset** from Kaggle, which includes multiple Excel sheets spanning FIFA 15 to FIFA 21.  
We will:
1. Download the dataset.
2. Inspect and clean each sheet.
3. Detect anomalies.
4. Merge all cleaned data into a single Excel file.


## 1. Set up and import

In [2]:
# --- Optional: install kagglehub in notebooks ---
# !pip install kagglehub

import os
from pathlib import Path
import pandas as pd
import kagglehub


## 2. Download the Dataset

We download the FIFA 21 dataset using `kagglehub`.  
The path to the downloaded files will be printed.


In [3]:
# Download dataset from kaggle
path = kagglehub.dataset_download("stefanoleone992/fifa-21-complete-player-dataset")
print("Path to dataset files:", path)


Path to dataset files: C:\Users\Admin\.cache\kagglehub\datasets\stefanoleone992\fifa-21-complete-player-dataset\versions\1


## 3. Load and Inspect the Excel File

We load the Excel file `Career Mode player datasets - FIFA 15-21.xlsx` and check its sheet names.


In [5]:
#read in the csv file as a pandas dataframe
fifa_21=pd.read_excel('Career Mode player datasets - FIFA 15-21.xlsx')
#Inspect sheet names
xls = pd.ExcelFile('Career Mode player datasets - FIFA 15-21.xlsx')
sheet_names = xls.sheet_names
print("Sheets found:", sheet_names)

Sheets found: ['FIFA 15', 'FIFA 16', 'FIFA 17', 'FIFA 18', 'FIFA 19', 'FIFA 20', 'FIFA 21']


## 4. Define Anomaly Check Helper

We define a function `check_anomalies()` to identify unrealistic values in player attributes such as age, height, weight, ratings, stats, and monetary columns.


In [8]:
# Anomaly check helper
def check_anomalies(df: pd.DataFrame) -> dict:
    issues = {}

    # 1. Age
    if 'age' in df.columns:
        issues['age'] = df[(df['age'] < 14) | (df['age'] > 60)]

    # 2. Height (cm)
    if 'height_cm' in df.columns:
        issues['height'] = df[(df['height_cm'] < 130) | (df['height_cm'] > 230)]

    # 3. Weight (kg)
    if 'weight_kg' in df.columns:
        issues['weight'] = df[(df['weight_kg'] < 30) | (df['weight_kg'] > 160)]

    # 4. Overall & Potential ratings
    if 'overall' in df.columns and 'potential' in df.columns:
        issues['rating'] = df[
            (df['overall'] < 0) | (df['overall'] > 99) |
            (df['potential'] < 0) | (df['potential'] > 99)
        ]

    # 5. Core stats (pace, shooting, etc.)
    stat_cols = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
    if all(col in df.columns for col in stat_cols):
        issues['stats'] = df[
            (df[stat_cols] < 0).any(axis=1) | (df[stat_cols] > 99).any(axis=1)
        ]

    # 6. Money columns
    money_cols = ['value_eur', 'wage_eur', 'release_clause_eur']
    existing_money = [c for c in money_cols if c in df.columns]
    if existing_money:
        issues['money'] = df[(df[existing_money] < 0).any(axis=1)]

    return issues

## 5. Define Sheet Cleaning Function

The function `clean_fifa_sheet()`:
- Strips whitespace from text columns.
- Converts date and monetary columns.
- Splits `work_rate` into `attack_work_rate` and `defense_work_rate`.
- Converts `player_traits` into a list.
- Removes duplicate players.
- Summarizes missing values and anomalies.


In [16]:
#Sheet cleaning function
def clean_fifa_sheet(sheet_name: str, xls_file: Path) -> pd.DataFrame:
    # Load the sheet
    df = pd.read_excel(xls_file, sheet_name=sheet_name)

    # 1. Strip whitespace in object (string) columns
    obj_cols = df.select_dtypes(include="object").columns
    df[obj_cols] = df[obj_cols].apply(lambda col: col.str.strip())

    # 2. Convert datetime columns (if present)
    for col in ['contract_valid_until', 'dob', 'joined']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # 3. Convert monetary columns to numeric
    for col in ['value_eur', 'wage_eur', 'release_clause_eur']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 4. Split work_rate into attack / defense
    if 'work_rate' in df.columns:
        # drop if already present (just in case)
        for c in ['attack_work_rate', 'defense_work_rate']:
            if c in df.columns:
                df.drop(columns=c, inplace=True)

        # convert to string before split to avoid errors from NaN
        split_cols = df['work_rate'].astype(str).str.split('/', n=1, expand=True)

        attack_col = split_cols[0].str.strip().str.lower()
        defense_col = split_cols[1].str.strip().str.lower() if split_cols.shape[1] > 1 else pd.NA

        idx = df.columns.get_loc('work_rate')
        df.insert(idx + 1, 'attack_work_rate', attack_col)
        df.insert(idx + 2, 'defense_work_rate', defense_col)

    # 5. Optional: list version of player traits
    if 'player_traits' in df.columns:
        df['player_traits_list'] = (
            df['player_traits']
              .fillna('')
              .apply(lambda x: x.split(', ') if x else [])
        )

    # 6. Drop duplicate players (sofifa_id)
    if 'sofifa_id' in df.columns:
        before = len(df)
        df = df.drop_duplicates(subset='sofifa_id')
        after = len(df)
        if before != after:
            print(f"{sheet_name}: dropped {before - after} duplicate sofifa_id rows.")



    # 8. Missing value summary
    total_missing = int(df.isna().sum().sum())
    print(f"{sheet_name}: total missing values = {total_missing}")
    missing_cols = df.isna().sum()
    if (missing_cols > 0).any():
        print((missing_cols[missing_cols > 0]).sort_values(ascending=False))

    # 9. Anomalies
    anomalies = check_anomalies(df)
    for k, v in anomalies.items():
        print(f"{sheet_name} - {k}: {len(v)} anomalies")

    return df

## 6. Clean All Sheets

We loop through all sheets, clean them with `clean_fifa_sheet()`, and store them in a dictionary.


In [17]:
#Clean all sheets & collect
cleaned_sheets = {}
for sheet in sheet_names:
    cleaned_sheets[sheet] = clean_fifa_sheet(sheet, 'Career Mode player datasets - FIFA 15-21.xlsx')

FIFA 15: total missing values = 201691
release_clause_eur      16155
mentality_composure     16155
loaned_from             15243
nation_position         15074
nation_jersey_number    15074
player_tags             14919
gk_speed                14380
gk_diving               14380
gk_kicking              14380
gk_handling             14380
gk_positioning          14380
gk_reflexes             14380
player_traits            9556
physic                   1775
shooting                 1775
pace                     1775
dribbling                1775
defending                1775
passing                  1775
joined                   1151
contract_valid_until      239
team_jersey_number        239
club_name                 239
league_name               239
league_rank               239
team_position             239
dtype: int64
FIFA 15 - age: 0 anomalies
FIFA 15 - height: 0 anomalies
FIFA 15 - weight: 0 anomalies
FIFA 15 - rating: 0 anomalies
FIFA 15 - stats: 0 anomalies
FIFA 15 - money: 0 ano

## 7. Save Cleaned Sheets

We save all cleaned sheets into a multi-sheet Excel file `fifa_15_21_cleaned_sheets.xlsx`.


In [18]:
# Save cleaned multi‑sheet Excel
cleaned_excel_name = "fifa_15_21_cleaned_sheets.xlsx"
with pd.ExcelWriter(cleaned_excel_name) as writer:  # no engine needed
    for sheet, df in cleaned_sheets.items():
        safe_sheet = sheet[:31]
        df.to_excel(writer, sheet_name=safe_sheet, index=False)
print(f"✅ Saved cleaned multi‑sheet Excel: {cleaned_excel_name}")


✅ Saved cleaned multi‑sheet Excel: fifa_15_21_cleaned_sheets.xlsx


## 8. Merge All Sheets into One

We add a `fifa_year` column to each sheet and merge them into a single DataFrame.  
The final merged file is saved as `fifa_2015_2021_merged_cleaned.xlsx`.


In [19]:
# Merge sheets into single DataFrame
merged_list = []
for sheet, df in cleaned_sheets.items():
    # Extract year from sheet name (e.g., "Fifa 15" -> 2015)
    year = ''.join(filter(str.isdigit, sheet))
    fifa_year = f"20{year}" if len(year) == 2 else year  # handle '15' -> '2015'
    df_copy = df.copy()
    df_copy['fifa_year'] = fifa_year
    merged_list.append(df_copy)

merged_df = pd.concat(merged_list, ignore_index=True)

# Save merged Excel
merged_excel_name = "fifa_2015_2021_merged_cleaned.xlsx"
merged_df.to_excel(merged_excel_name, index=False)
print(f"✅ Saved merged Excel: {merged_excel_name}")

✅ Saved merged Excel: fifa_2015_2021_merged_cleaned.xlsx
